# Data Processing

In [83]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
import seaborn as sns
import matplotlib.pyplot as plt

from model import train_and_save_model

In [84]:
RAW_DATA_DIR = Path('C:\\Users\\loicc\\OneDrive - Efrei\\Bureau\\COURS\\M2\\S9\\Machine Learning in Production\\Data Pipeline\\mlops-nba')

# Replace 'nom_du_fichier.csv' with the name of your CSV file
filename = RAW_DATA_DIR / '2023-2024 NBA Player Stats - Regular.csv'
print(f"Running on file: {filename}")
players_df = pd.read_csv(filename, sep=',', encoding='Windows-1252')

# Add a new 'file' column to the DataFrame
players_df['filename'] = filename.name

Running on file: C:\Users\loicc\OneDrive - Efrei\Bureau\COURS\M2\S9\Machine Learning in Production\Data Pipeline\mlops-nba\2023-2024 NBA Player Stats - Regular.csv


In [85]:

def save_dataframe_as_parquet(dataframe, path, filename, loaded_files=None):
    # Vérifier si loaded_files est fourni et non vide
    if loaded_files is not None and len(loaded_files) > 0:
        number_of_files = len(loaded_files)
        # Modifier le nom du fichier pour inclure ce nombre
        modified_filename = filename.replace('.parquet', f'_{number_of_files}.parquet')
    else:
        # Garder le nom de fichier original si loaded_files n'est pas fourni
        modified_filename = filename

    # Chemin complet du fichier
    full_path = os.path.join(path, modified_filename)

    # Sauvegarder le DataFrame en fichier Parquet
    dataframe.to_parquet(full_path, index=False)

In [86]:
save_dataframe_as_parquet(players_df, 'C:\\Users\\loicc\\OneDrive - Efrei\\Bureau\\COURS\\M2\\S9\\Machine Learning in Production\\Data Pipeline\\mlops-nba\\stockage', 'players_raw.parquet')

In [87]:
def process_players_data(players):
    # Assurer que le DataFrame n'est pas vide
    if players.empty:
        raise ValueError("Le DataFrame 'players' est vide.")

    # Ajouter la colonne EFF
    players["EFF"] = players.PTS + players.TRB + players.AST + players.STL + players.BLK - \
                     (players.FGA - players.FG) - (players.FTA - players.FT) - players.TOV

    # Calculer les statistiques d'âge et de points
    ages = players.Age.describe().round(decimals=1)
    points = players.PTS.describe().round(decimals=1)

    # Définir les critères pour les futurs superstars
    young_age = ages["25%"]
    futur_super_star_def = f"(EFF >= 12) & (PTS >= 15) & (Age <= {young_age})"
    players.query(futur_super_star_def).sort_values("EFF", ascending=False).sort_values(["Age", "EFF"], ascending=True)

    # Calculer le pourcentage de tir
    players['TS%'] = np.where((2 * (players['FGA'] + 0.44 * players['FTA'])) != 0, 
                              players['PTS'] / (2 * (players['FGA'] + 0.44 * players['FTA'])), 0)

    players["Pos"] = players["Pos"].replace("C-PF", "C")
    # Mapper les positions des joueurs
    players["position"] = players.Pos.map({"PG": "Backcourt", "SG": "Backcourt", 
                                           "SF": "Wing", "SF-PF": "Wing", 
                                           "PF": "Big", "C": "Big"})


    # Retourner le DataFrame traité
    return players

# Utilisation de la fonction
# players = pd.read_csv('chemin_du_fichier.csv', sep=',', encoding='Windows-1252')
# players_processed = process_players_data(players)


In [88]:
players_df = process_players_data(players_df)

stockage

In [89]:
save_dataframe_as_parquet(players_df, 'C:\\Users\\loicc\\OneDrive - Efrei\\Bureau\\COURS\\M2\\S9\\Machine Learning in Production\\Data Pipeline\\mlops-nba\\stockage', 'players_final.parquet')

In [90]:
train_and_save_model(players_df)

RMSE for PTS prediction: 0.8944068476319434
RMSE for FG% prediction: 0.059614850270106934
Modèle sauvegardé sous : C:\Users\loicc\OneDrive - Efrei\Bureau\COURS\M2\S9\Machine Learning in Production\Data Pipeline\mlops-nba\Model\model_4.joblib


# Append DATA

In [91]:
loaded_files = {}

In [92]:
# # Dictionnaire pour suivre les fichiers déjà lus
# loaded_files = {}

# Chemin vers le dossier de données
RAW_DATA_DIR = Path('C:\\Users\\loicc\\OneDrive - Efrei\\Bureau\\COURS\\M2\\S9\\Machine Learning in Production\\Data Pipeline\\mlops-nba\\Append Data')

# Parcourir tous les fichiers CSV dans le dossier
for file_path in RAW_DATA_DIR.glob('*.csv'):
    file_name = file_path.name

    # Vérifier si le fichier a déjà été lu
    if file_name not in loaded_files:
        # Lire les données du fichier CSV
        new_data = pd.read_csv(file_path, sep=';', encoding='Windows-1252')

        # Ajouter une colonne 'filename' avec le nom du fichier
        new_data['filename'] = file_name

        # Sauvegarder la taille actuelle du DataFrame
        current_length = len(players_df)

        # Concaténer avec le DataFrame principal
        players_df = pd.concat([players_df, new_data], ignore_index=True)

        # Calculer le nombre de lignes ajoutées
        new_lines_added = len(players_df) - current_length
        if new_lines_added > 0:
            print(f"{new_lines_added} nouvelles lignes ajoutées depuis {file_name}.")

        # Marquer le fichier comme lu
        loaded_files[file_name] = True


812 nouvelles lignes ajoutées depuis 2021-2022 NBA Player Stats - Regular.csv.
679 nouvelles lignes ajoutées depuis 2022-2023 NBA Player Stats - Regular.csv.


In [93]:
players_df['Pos'].value_counts()

Pos
SG       492
SF       399
PF       382
PG       382
C        357
SG-PG      6
SF-SG      6
SG-SF      5
PF-SF      4
C-PF       2
SF-PF      1
PG-SG      1
Name: count, dtype: int64

In [94]:
save_dataframe_as_parquet(players_df, 'C:\\Users\\loicc\\OneDrive - Efrei\\Bureau\\COURS\\M2\\S9\\Machine Learning in Production\\Data Pipeline\\mlops-nba\\stockage', 'players_append.parquet', loaded_files)

In [95]:
# Utiliser la fonction pour entraîner et sauvegarder le modèle
train_and_save_model(players_df)

RMSE for PTS prediction: 0.42638473087751383
RMSE for FG% prediction: 0.0602597441107587
Modèle sauvegardé sous : C:\Users\loicc\OneDrive - Efrei\Bureau\COURS\M2\S9\Machine Learning in Production\Data Pipeline\mlops-nba\Model\model_5.joblib
